In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pip

In [1]:
!pip install selenium
!pip install bs4

from bs4 import BeautifulSoup
from selenium import webdriver
from collections import defaultdict

import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')

driver = webdriver.Chrome(options=options)

x_descriptions, y_descriptions = set(), set()
x_company_dict, y_company_dict = defaultdict(list), defaultdict(list)

batches = [season + year for year in [str(i).zfill(2) for i in range(5, 24)] for season in ["S", "W"]]

for batch in batches:
    
    driver.get(f"https://www.ycombinator.com/companies?batch={batch}")
    
    # Get the current scroll height
    last_height = driver.execute_script("return Math.max( document.body.scrollHeight, document.body.offsetHeight, document.documentElement.clientHeight, document.documentElement.scrollHeight, document.documentElement.offsetHeight );")
    new_height = 0
    
    while True:
        # Scroll down to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for the website to load new content
        time.sleep(0.1)

        # Get the new scroll height
        new_height = driver.execute_script("return Math.max( document.body.scrollHeight, document.body.offsetHeight, document.documentElement.clientHeight, document.documentElement.scrollHeight, document.documentElement.offsetHeight );")

        # If the new height is the same as the last height, it means the end of the page has been reached
        if new_height == last_height:
            break

        # Update the last height
        last_height = new_height
    
    # Get the HTML source code of the page
    html = driver.page_source

    # Parse the HTML using Beautiful Soup
    soup = BeautifulSoup(html, "html.parser")
    
    # Get all company names and descriptions
    raw_descriptions = soup.find_all("span", class_="OCTUb4j7DGmBqnUsAzVD")
    raw_companies = soup.find_all("span", class_="CBY8yVfV0he1Zbv9Zwjx")
    
    # Convert them to lists of strings
    descriptions = [str(raw_description.text) for raw_description in list(raw_descriptions)]
    companies = [str(raw_company.text) for raw_company in list(raw_companies)]
    
    # Remove all descriptions and companies that are not "X for Y"
    to_remove = []
    for idx, description in enumerate(descriptions):
        if " for " not in description:
            to_remove.append(idx)

    for idx in sorted(to_remove, reverse=True):
        del descriptions[idx]
        del companies[idx]
    
    # Split all descriptions at the "for"
    batch_x_descriptions, batch_y_descriptions = [description.split(" for ")[0] for description in descriptions], [description.split(" for ")[1] for description in descriptions]
    x_descriptions.extend(batch_x_descriptions)
    y_descriptions.extend(batch_y_descriptions)
    
    # Map company name to x and y of a description
    for idx, description in enumerate(descriptions):
        company = companies[idx]
        x_description, y_description = batch_x_descriptions[idx], batch_y_descriptions[idx]
        x_company_dict[x_description].append(company)
        y_company_dict[y_description].append(company)
        print(f"{x_description} for {y_description} ({x_company_dict[x_description]} x {y_company_dict[y_description]})")

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home
